In [1]:
!git clone https://github.com/ramirosi/Intent-Guided-Fine-Tuning.git
%cd /content/Intent-Guided-Fine-Tuning

Cloning into 'Intent-Guided-Fine-Tuning'...
remote: Enumerating objects: 476, done.
remote: Counting objects: 100% (476/476), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 476 (delta 165), reused 476 (delta 165), pack-reused 0 (from 0)
Receiving objects: 100% (476/476), 3.02 MiB | 11.25 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/Intent-Guided-Fine-Tuning


# Stage 1

Stage 1: labeling focused on using a closed-source LLM to label messages inside conversations with intents.

In [9]:
%cd "/content/Intent-Guided-Fine-Tuning/Stage 1/"

/content/Intent-Guided-Fine-Tuning/Stage 1


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.8 MB/s eta 0:00:00


## Toy dataset

In [3]:
import pandas as pd

Let's read the dummy data containing 25 customer service conversations similar to the ones in used in the thesis. Each message within the conversations has already been labeled with context-aware intents (in this case not human-labeled, but also labeled by GPT-4o).



In [11]:
label_intent_df = pd.read_csv("toy_conversations.csv")
print(len(label_intent_df))
label_intent_df

239


,conversation_number,conversation,index,intent
0,0,"customer: Hi, I'm having trouble with my loyal...",0,report_problem
1,0,assistant: Hello! Sorry to hear that. Can you ...,1,ask_clarification
2,0,customer: The card isn't being recognized at c...,2,describe_issue
3,0,"assistant: I understand, that must be frustrat...",3,suggest_troubleshooting
4,0,"customer: Yes, I tried both. Still doesn't work.",4,confirm_attempted_solution
...,...,...,...,...
234,25,assistant: That would explain it. I’ll send yo...,3,offer_solution
235,25,"customer: Great, thank you.",4,accept_solution
236,25,assistant: The new card will arrive within 5 w...,5,confirm_action
237,25,"customer: Perfect, much appreciated.",6,express_gratitude


## GIFA

Lets run the generative intent finding algorithm (GIFA). GIFA iteratively calls an LLM (GPT-4o) to label each message with intents per conversation, and keeps an expanding known intent database. We provided a simulated GIFA run (toy_intents.csv) that used GPT-4o inside GIFA run on the dummy dataset.

GIFA also uses a intent search setup, selecting the most relevant intents for a new conversation to label. This search is done using embeddings: with a new message, we want to find the most similar historical messages and extract their intents and add them to the prompt. To find the most similar messages embeddings are needed. In our simulation these embeddings can be found in GIFA->retrieval->embeddings.csv and are create with the ADA-003 model.

In [12]:
from GIFA.generate_intent_full import IntentGenerator
from GPFA.update_prompt import IntentPromptUpdater

dynamic_prompt = '''
                  You are analyzing a customer service conversation to identify the underlying intentions behind each message.
                  Assign a structured intent label to each message.

                  - Use an existing intent label if it already covers the message.
                  - Create a new intent only if the message introduces a distinct issue not covered by existing intents.
                 '''

updater = IntentPromptUpdater(dynamic_prompt)
static_prompt = updater.create_base_static_prompt()

conv_df = label_intent_df.drop("intent", axis=1)

api_time = 3 # Time between API calls
num_samples = 25 # Number of available samples (conversations)
sampling_method = "example" # Search method for finding relevant intents for a new conversation
top_k = 2 # Amount of relevant intents retrieved per new message in a new conversation (these are added to the prompt)
top_n = 1 # Amount of examples added to each intent name within the prompts (only when sampling method = example!)
search = True # Specify if it should search for relevant intents or add just every intent found yet

generator = IntentGenerator(static_prompt, api_time=api_time, n_samples=num_samples, method = sampling_method,
                                     top_k=top_k, top_n=top_n, search=True, show_tqdm=True, dummy_mode=True, dummy_file="toy_intents.csv")
full_intent_df, unique_intents = generator.run_intent_generation(conv_df)
full_intent_df.head()

INFO:GIFA.generate_intent_full:✅ IntentGenerator initialized
INFO:GIFA.generate_intent_full:🚀 Starting intent generation...
INFO:GIFA.generate_intent_full:📂 Dummy mode: loading saved intents from toy_intents.csv


index,intent,count,messages
i64,str,i64,str
0,"""report_loyalty_card_issue""",0,"""customer: Hi, I'm having troub…"
1,"""request_issue_details""",1,"""assistant: Hello! Sorry to hea…"
2,"""explain_issue""",2,"""customer: The card isn't being…"
3,"""suggest_troubleshooting""",3,"""assistant: I understand, that …"
4,"""confirm_troubleshooting_attemp…",4,"""customer: Yes, I tried both. S…"


To evaluate the intents generated by the GIFA algorithm, we compare them to the "labeled" toy dataset, and compute clustering metrics and find examples that went wrong.

In [13]:
from GPFA.evaluate_intent import encode_intent_labels, merge_intent_dfs
from GPFA.evaluate_intent import IntentClusteringEvaluator

df_merged = merge_intent_dfs(label_intent_df, full_intent_df)
true_clusters, predicted_clusters = encode_intent_labels(df_merged)
evaluator = IntentClusteringEvaluator(true_clusters, predicted_clusters, list(df_merged["message"]),
                                    list(df_merged["intent_true"]), list(df_merged["intent_predicted"]))
results = evaluator.evaluate()

print(results)

{'Clustering Metrics': {'Purity': np.float64(0.5591836734693878), 'NCA': np.float64(0.8609629449001823), 'AMI': np.float64(0.4165876897866946), 'pairwise_precision': 0.29203084832904885, 'pairwise_recall': 0.6147186147186147, 'pairwise_F1': 0.39595677936563256, 'ARI': 0.36953093382976276, 'NMI': np.float64(0.7795144814955736), 'FMI': np.float64(0.42369422764528525)}, 'Errors': {'Interesting Examples': [('Message: customer: Yes please, that would be great. AND customer: Yes, please. should have the same intent: accept_solution', 'false_split'), ('Message: customer: No, that’s it. AND assistant: Anytime! Have a good day. should have had seperate intents: end_request AND close_conversation', 'false_merge'), ('Message: assistant: Great. Have a wonderful day! AND assistant: Happy to help! should have the same intent: close_conversation', 'false_split'), ('Message: assistant: Sometimes banks decline transactions for security. Could you try again or use PayPal instead? AND assistant: Sure, I 

## GPFA

GIFA's performance is highly dependent on the prompt used. To optimize the prompt without human interference, we introduced the generative intent finding algorithm (GIFA). GIFA iteratively optimizes a prompt through feedback based on the reported clustering metrics and examples. GIFA can use multiple background search algorithms, which guide the process of which prompts to select during optimization.

Four of them are included: greedy search; random search; SEE and Hyperband-SEE.

Each optimization has been run using GPT-4o and the results were saved, such that other people without LLM access can still run the code.

- Random search simulation can be found in the rpo_toy folder: each prompt found is evaluated using GIFA, and the results for those are in the GIFA folder; the prompts folder shows the different prompts found through the process, and history provided more meta-data on the process.
- Greedy search works the same.

- SEE works the same, but also some embeddings (ADA-003) are stored, these embeddings are used when using a semantic-wise crossover round.

- Hyperband-SEE is more nested. Each round inside Hyperband-SEE runs SEE itself. So for each round the SEE run is saved with inside of it the GIFA runs.

### Random prompt search

In [14]:
from GPFA.greedy_prompt_finding import greedy_prompt_optimizer

type = "random" # Baseline type random or greedy
train_df = conv_df # Can have a train set and validation set. Train is used for feedback.
val_df = conv_df
population_size = 10 # Maximum amount of prompts to be tested
R_max = 25 # Maximum amount of conversations to be evaluated on
method = "name" # Sampling method
init_temperature = 0.7 # Temperature used for feedback application
top_k = 2 # Intent search depth
lagged_feedback = True # True: first call the LLM to create feedback, then in a second call apply this feedback to the prompt, False: direct
show_tqdm = True # Show progress bar

rpo = greedy_prompt_optimizer(dynamic_prompt=dynamic_prompt, train_df=train_df, val_df=val_df, label_intent_df=label_intent_df, population_size=population_size,
                              num_samples=R_max, method=method, top_k=top_k, top_n=2, init_temperature=init_temperature, lagged_feedback=lagged_feedback, type=type,
                              show_tqdm=show_tqdm, dummy_mode=True, dummy_folder="rpo_toy")
history = rpo.run_gpo()

########################################
###### Currently at Iteration 0 ######
########################################
Updated Prompt: 
                  You are analyzing a customer service conversation to identify the underlying intentions behind each message. 
                  Assign a structured intent label to each message.  

                  - Use an existing intent label if it already covers the message.  
                  - Create a new intent only if the message introduces a distinct issue not covered by existing intents.  
                 
########################################
###### Currently at Iteration 1 ######
########################################
Updated Prompt: You are analyzing a customer service conversation to identify the underlying intentions behind each message. Assign a structured intent label to each message, using the following guidelines:

1. **Intent Definitions**: Clearly defined intent labels with examples are provided. For instance, "express_

In [15]:
history["ami_x_nca"].max()

0.5266810320159518

### Greedy prompt search

In [16]:
type = "greedy"


rpo = greedy_prompt_optimizer(dynamic_prompt=dynamic_prompt, train_df=train_df, val_df=val_df, label_intent_df=label_intent_df, population_size=population_size,
                              num_samples=R_max, method=method, top_k=top_k, top_n=2, init_temperature=init_temperature, lagged_feedback=lagged_feedback, type=type,
                              show_tqdm=show_tqdm, dummy_mode=True, dummy_folder="gpo_toy")
history = rpo.run_gpo()

########################################
###### Currently at Iteration 0 ######
########################################
Updated Prompt: 
                  You are analyzing a customer service conversation to identify the underlying intentions behind each message. 
                  Assign a structured intent label to each message.  

                  - Use an existing intent label if it already covers the message.  
                  - Create a new intent only if the message introduces a distinct issue not covered by existing intents.  
                 
########################################
###### Currently at Iteration 1 ######
########################################
Updated Prompt: You are analyzing a customer service conversation to identify the underlying intentions behind each message. Assign a structured intent label to each message, ensuring clarity, consistency, and contextual analysis. 

- Use an existing intent label if it already covers the message.
- Create a new int

In [17]:
history["ami_x_nca"].max()

0.5600614241910931

### SEE

In [18]:
from GPFA.see_algorithm import SEE

population_size = 5 # Amount of prompts in each phase
train_df = conv_df
val_df = conv_df
val_samples = 25
K1 = 2 # Amount of feedback rounds
K2 = 0 # Amount of crossover rounds

# K2 = 2
crossover_types = ["ADA", "error"] # Crossover round types, either semantic-wise (ADA) or error-wise (error)

see = SEE(train_df, val_df, label_intent_df, population_size=population_size, method = method,
                                    search_method = "semantic", top_n = 2, top_k = 2, init_temperature=0.7,
                                    val_samples=val_samples, K1=K1, feedback_samples=15, K2=K2, crossover_types = crossover_types,
                                    lagged_feedback=lagged_feedback, show_tqdm=True, dummy_mode=True, dummy_folder="see_toy" )
history = see.run()

Feedback loop: 0
Feedback loop: 1


In [20]:
see.history["ami_x_nca"].max()

0.394566658007283

### Hyperband-SEE

In [21]:
from GPFA.hyperband_scheduler import find_general_hyperband_schedule, time_to_budget

eta_values = [2, 3, 5] # Set halving factors to be tested
hours = 1 # Set time that can be used as budget 11
B = time_to_budget(hours) # Calculate the amount of total evaluations that can be done with the time budget
R_min = 4 # Set minimum amount of evaluation samples 10
R_max = 25 # Set maximum amount of evaluation samples
n_min = 1 # Set minimum amount of prompts needed to start PhaseEvo 4

# Create the schedules for the chosen hyperparameters
schedules = find_general_hyperband_schedule(B, eta_values, R_min, R_max, n_min)
schedules

[{'eta': 2,
  'rounds': 3,
  'x_list': [8.0, 4.0, 2.0],
  'R_list': [4, 8, 16],
  'multiplicity': [3, 2, 1],
  'total_cost': 768.0},
 {'eta': 3,
  'rounds': 2,
  'x_list': [18.0, 6.0],
  'R_list': [4, 12],
  'multiplicity': [2, 1],
  'total_cost': 864.0},
 {'eta': 5,
  'rounds': 2,
  'x_list': [15.0, 3.0],
  'R_list': [4, 20],
  'multiplicity': [2, 1],
  'total_cost': 720.0}]

In [23]:
from GPFA.hyperband_see import hb_see

hyp = hb_see(train_df, val_df, label_intent_df, schedules[0], method=method, search_method="semantic",
                      top_k=top_k, init_temperature=init_temperature, feedback_samples=15, K1=2, K2=0,
                      crossover_type=crossover_types, lagged_feedback=lagged_feedback, show_tqdm=show_tqdm, filename=None,
                      dummy_mode=True, dummy_root="hyperband_see")

history = hyp.run_hyperband()

Bracket 1/3:  67%|██████▋   | 2/3 [00:00<00:00,  6.83it/s]

Finished Hyperband (bracket 1, round 1) in 0.2s | pop=8, val_samples=4
Finished Hyperband (bracket 1, round 2) in 0.1s | pop=4, val_samples=8


Bracket 1/3: 100%|██████████| 3/3 [00:00<00:00,  7.38it/s]


Finished Hyperband (bracket 1, round 3) in 0.1s | pop=2, val_samples=16


Bracket 2/3:  50%|█████     | 1/2 [00:00<00:00,  9.21it/s]

Finished Hyperband (bracket 2, round 1) in 0.1s | pop=4, val_samples=8


Bracket 2/3: 100%|██████████| 2/2 [00:00<00:00,  8.95it/s]


Finished Hyperband (bracket 2, round 2) in 0.1s | pop=2, val_samples=16


Bracket 3/3: 100%|██████████| 1/1 [00:00<00:00,  8.61it/s]

Finished Hyperband (bracket 3, round 1) in 0.1s | pop=2, val_samples=16


In [24]:
# Scores found in the last round

history[history["round"] == 3]["ami_x_nca"].max()

0.5906197155356346

# Stage 2

Stage 2: modeling focussed on fine-tuning an open-source LLM with the intent-labeled dataset from stage 1. Below we give some example code to replicate the intent-guided CoT fine-tuning.

MAKE SURE TO TURN ON A GPU ENVIRONMENT WHEN RUNNING THIS STAGE! (At least 15 GB VRAM)

## Imports

In [25]:
import torch

# Get PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available")
    print("CUDA version:", torch.version.cuda)
    print("GPU device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available")

PyTorch version: 2.8.0+cu126
CUDA is available
CUDA version: 12.6
GPU device name: Tesla T4


In [26]:
!pip install "unsloth[cu126-torch280]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [27]:
!pip install rouge_score
!pip install bert_score
!pip install sacrebleu
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6b5fb75954d516d4256595d5e87da7517f55783734c1bbb1d69110ef80a9e565
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00


In [28]:
import re
import torch
import random
import pandas as pd
import numpy as np
from datasets import Dataset, Features, Sequence, Value, DatasetDict
import torch.nn.functional as F
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer
import unittest
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import f1_score
import gc
import json
from collections import deque
from tqdm.auto import tqdm
from typing import List, Tuple
import os
import ast
from tqdm.auto import tqdm
from transformers import EarlyStoppingCallback

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Datasets

In [29]:
# ───────────────────── helpers ────────────────────────────────────
ROLE_PATTERN = re.compile(r"^(customer|assistant|klant|medewerker):\s*(.*)", re.I)

def split_role_text(msg: str) -> Tuple[str, str]:
    m = ROLE_PATTERN.match(str(msg).strip())
    if not m:
        raise ValueError(f"Bad role prefix in: {msg!r}")
    role_raw, body = m.groups()
    role     = "user" if role_raw.lower() in ("customer", "klant") else "assistant"
    visible  = "klant" if role == "user" else "medewerker"
    return role, f"{visible}: {body.strip()}"

def _collapse_with_idx(df: pd.DataFrame, col: str):
    cols = ["role", col, "intent"]
    out, cur, buf_t, buf_i, last = [], None, [], [], None
    for idx, role, text, intent in df[cols].itertuples(index=True, name=None):
        if role != cur and buf_t:
            out.append((cur, buf_t, buf_i, last))
            buf_t, buf_i = [], []
        cur, last = role, idx
        buf_t.append(str(text))
        buf_i.append("" if pd.isna(intent) else str(intent))
    if buf_t:
        out.append((cur, buf_t, buf_i, last))
    return out
# ───────────────────── main builder ───────────────────────────────
def build_instruction_dataframe(
    df: pd.DataFrame,
    tokenizer,
    system_prompt: str,
    max_seq_len: int = 2048,
    output_mode: str = "intent+response",
    use_rag: bool = False,
    rag_col: str = "retrieved_passages",
    rag_top_k: int = 3,
):
    """
    • No explicit <|im_start|>system block.
    • system_prompt is prepended to the FIRST user message, or put in a
      synthetic user stub if the assistant speaks first.
    • RAG passages (if any) are injected ABOVE the user turn they belong to,
      inside the same user message.
    """
    dfc = df.copy().reset_index(drop=True)

    if use_rag and rag_col in dfc.columns and isinstance(dfc[rag_col].iloc[0], str):
        dfc[rag_col] = dfc[rag_col].apply(
            lambda x: ast.literal_eval(x) if pd.notna(x) else []
        )

    dfc[["role", "prefixed_text"]] = pd.DataFrame(
        [split_role_text(m) for m in dfc["conversation"]], index=dfc.index
    )
    dfc = dfc.sort_values(["conversation_number", "message_number"])

    samples = []

    for _, group in tqdm(dfc.groupby("conversation_number", sort=False),
                         desc="Building dataset"):
        turns    = _collapse_with_idx(group, col="prefixed_text")
        history  : List[dict] = []
        sys_done = False

        for role, texts, intents, idx in turns:
            content = "\n".join(texts)

            # ---------------- USER TURN ----------------
            if role != "assistant":
                # inject system_prompt on the first user turn
                if not sys_done:
                    content = f"{system_prompt}\n\nConversation History:\n{content}" if system_prompt else content
                    sys_done = True

                # optional RAG injection happens HERE
                if use_rag and rag_col in group.columns:
                    rag_passages = group.loc[idx, rag_col]
                    if isinstance(rag_passages, list) and rag_passages:
                        rag_txt = "\n\n".join(rag_passages[:rag_top_k])
                        content = (
                            "--- RELEVANT KNOWLEDGE ---\n" + rag_txt
                            + "\n\n--- END KNOWLEDGE ---\n\n" + content
                        )

                history.append({"role": "user", "content": content})
                continue

            # if assistant starts the conversation → create stub user w/ system
            if not sys_done:
                stub = {"role": "user", "content": system_prompt}
                history.append(stub)
                sys_done = True

            # ------------- ASSISTANT TURN --------------
            intents_str = "\n".join(f"- {i}" for i in intents if i)
            if output_mode == "response":
                assistant_content = content
            elif output_mode == "intent+response":
                assistant_content = (
                    f"Intents:\n{intents_str}"
                    f"\nResponses:\n{content}"
                )
            else:
                raise ValueError(f"Bad output_mode: {output_mode}")

            prompt_msgs = history
            full_msgs   = prompt_msgs + [{"role": "assistant", "content": assistant_content}]

            if len(tokenizer.apply_chat_template(full_msgs)) >= max_seq_len:
                continue  # skip over-long sample

            samples.append(
                {
                    "index": idx,
                    "training_text": tokenizer.apply_chat_template(
                        full_msgs, tokenize=False, add_generation_prompt=False
                    ),
                    "prompt_text": tokenizer.apply_chat_template(
                        prompt_msgs, tokenize=False, add_generation_prompt=True
                    ),
                    "assistant_intents_only": "\n".join(i for i in intents if i),
                    "assistant_response_raw": content,
                }
            )

            history.append({"role": "assistant", "content": content})

    if not samples:
        return Dataset.from_list([]), pd.DataFrame()

    df_out = pd.DataFrame(samples).set_index("index")
    df_out = dfc.loc[df_out.index].join(df_out)
    ds_out = Dataset.from_pandas(
        df_out[["training_text"]].rename(columns={"training_text": "text"})
    )

    return ds_out, df_out.reset_index(drop=True)

In [30]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
import gc
from sklearn.metrics import f1_score
import re
from typing import Tuple, List, Optional
from datasets import Dataset
from tqdm.auto import tqdm
import json
import evaluate

def split_conversations(df, conversation_col='conversation_number', val_ratio=0.1, method='sequential', seed=42):

    conv_ids = df[conversation_col].unique()

    if method == 'random':
        np.random.seed(seed)
        conv_ids = np.random.permutation(conv_ids)
    elif method == 'sequential':
        conv_ids = sorted(conv_ids)  # assumes time-ordered
    else:
        raise ValueError("Method must be 'sequential' or 'random'")

    split_idx = int(len(conv_ids) * (1 - val_ratio))
    train_ids = conv_ids[:split_idx]
    val_ids = conv_ids[split_idx:]

    train_df = df[df[conversation_col].isin(train_ids)]
    val_df = df[df[conversation_col].isin(val_ids)]

    return train_df.reset_index(drop=True), val_df.reset_index(drop=True)

ROLE_RE = re.compile(r'^(customer|assistant|klant|medewerker):', re.I)

def filter_mono_role_convs(df: pd.DataFrame,
                           msg_col: str = "conversation",
                           id_col: str  = "conversation_number"
                          ) -> Tuple[pd.DataFrame, int, int]:

    roles = df[msg_col].str.extract(ROLE_RE, expand=False).str.lower()
    df_role = df.assign(_role=roles)

    role_sets = (df_role.groupby(id_col)["_role"].apply(lambda s: set(s.dropna())))

    valid_conv_ids = role_sets[
    role_sets.apply(lambda x: bool({"customer", "assistant", "klant", "medewerker"} & x) and len(x) > 1)
    ].index

    df_filtered = df_role[df_role[id_col].isin(valid_conv_ids)].drop(columns="_role")

    kept    = len(valid_conv_ids)
    dropped = role_sets.size - kept
    print(f"kept {kept} conversations, dropped {dropped} (mono-role)")
    return df_filtered

def setup_data(path="data/sample_25k_train.csv", val_ratio=0.1):

    df = pd.read_csv(path)
    df = df.rename(columns={"index": "message_number"})

    try:
        df.drop("Unnamed: 0", axis=1)
    except:
        pass

    df = df.sort_values(["conversation_number", "message_number"]).reset_index(drop=True)

    bad_conversations = df[df["intent"].isnull()]["conversation_number"].unique()
    df = df[~df["conversation_number"].isin(bad_conversations)]

    train_df, val_df = split_conversations(df.reset_index(drop=True), val_ratio=val_ratio)

    train_df = filter_mono_role_convs(train_df)
    val_df = filter_mono_role_convs(val_df)

    train_df = train_df.sort_values(["conversation_number", "message_number"]).reset_index(drop=True)
    val_df = val_df.sort_values(["conversation_number", "message_number"]).reset_index(drop=True)

    return train_df, val_df

In [32]:
train_df, val_df = setup_data(path="/content/Intent-Guided-Fine-Tuning/Stage 2/toy_conversations.csv", val_ratio=0.3)

kept 18 conversations, dropped 0 (mono-role)
kept 8 conversations, dropped 0 (mono-role)


## Setup QLoRA Qwen3 1.7B model

In [33]:
def create_model(model_name = "unsloth/Qwen3-1.7B-bnb-4bit", max_seq_length = 2048):

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        load_in_4bit=True,
        dtype=None
    )

    tokenizer.padding_side = "left"

    return model, tokenizer

In [34]:
max_seq_len = int(2048) # Max sequence length, all large saples are deleted
r = 16
lora_alpha = 32

model_name = "Qwen3-1.7B-bnb-4bit"
model_url = "unsloth/Qwen3-1.7B-bnb-4bit"

per_device_train_batch_size = 16
per_device_eval_batch_size = 64
max_steps = 2000
learning_rate = 2e-4

In [35]:
model, tokenizer = create_model(model_name=model_url, max_seq_length=2048)

==((====))==  Unsloth 2025.9.1: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Setting up the fine-tuning data format. In this example, we can either directly predict the assistant reply, or do the CoT format: first intent planning then replies. If RAG has to be performed, add knowledge to the toy dataset.

In [36]:
# Use the prompt below if you want only responses
# system_prompt = str("You are a helpful assistant. Given the conversation, write the next assistant reply.")

system_prompt = str('''You are an expert customer service assistant for our company. You must always follow a strict two-step process to reply.
1.  **Intents:** First, you must determine the correct set of response intents needed. Write these on a new line starting with 'Intents:'.
2.  **Responses:** After a blank line, you must write one or multiple full, polite, and helpful responses to the customer, starting with 'Responses:'.''')


print("Building training set")
ds_train, df_train_f = build_instruction_dataframe(
    df=train_df, # Your training data, which MUST have the 'retrieved_passages' column
    tokenizer=tokenizer,
    system_prompt=system_prompt,
    max_seq_len=max_seq_len,
    output_mode="intent+response", #or "response"
    use_rag=False,
    rag_top_k=3   # Specify how many retrieved articles to use
)

print("\nBuilding validation set")
ds_val, df_val_f = build_instruction_dataframe(
    df=val_df, # Your validation data with retrieved passages
    tokenizer=tokenizer,
    system_prompt=system_prompt,
    max_seq_len=max_seq_len,
    output_mode="intent+response", # or "response"
    use_rag=False, # <-- Also set this to True for the validation set
    rag_top_k=3
)

Building training set


Building dataset:   0%|          | 0/18 [00:00<?, ?it/s]


Building validation set


Building dataset:   0%|          | 0/8 [00:00<?, ?it/s]

## Model Training

In [37]:
def model_config(train_ds, valid_ds, model, tokenizer, max_seq_length = 2048,
                 per_device_train_batch_size = 4, per_device_eval_batch_size=4, max_steps = 2000, learning_rate = 2e-4):

    cfg = SFTConfig(

        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        max_steps = max_steps,
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,

        warmup_ratio=0.03,
        lr_scheduler_type="cosine",

        eval_strategy="steps",
        eval_steps = 1,

        remove_unused_columns=False,
        seed=42,

        metric_for_best_model="eval_loss",
        report_to=[],
        greater_is_better=False
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        args=cfg,
    )

    early_stop = EarlyStoppingCallback(
        early_stopping_patience = 2,      # “2 evals in a row”
        early_stopping_threshold = 0.005,   # “no improvement”
    )
    trainer.add_callback(early_stop)

    return trainer

def lora_config(model, r=16, lora_alpha=32):
    model = FastLanguageModel.get_peft_model(
    model,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    )

    return model

In [38]:
model = lora_config(model, r=r, lora_alpha=lora_alpha)
trainer = model_config(ds_train, ds_val, model, tokenizer,
                       max_seq_length = max_seq_len,
                       per_device_train_batch_size = per_device_train_batch_size,
                       per_device_eval_batch_size=per_device_eval_batch_size,
                       max_steps = 10,
                       learning_rate = learning_rate)
trainer.train()

Unsloth 2025.9.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/88 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/28 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 88 | Num Epochs = 4 | Total steps = 10
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 2 x 1) = 32
 "-____-"     Trainable parameters = 17,432,576 of 1,738,007,552 (1.00% trained)
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
1,4.563500,4.682938
2,4.589700,4.682938
3,4.589200,4.682938


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=3, training_loss=4.580824216206868, metrics={'train_runtime': 36.1256, 'train_samples_per_second': 8.858, 'train_steps_per_second': 0.277, 'total_flos': 289295195701248.0, 'train_loss': 4.580824216206868, 'epoch': 1.0})

## Inference

In [39]:
def _infer_single_batch(
    batch_df: pd.DataFrame,
    model,
    tokenizer,
    *,
    max_new_tokens: int = 20,
    phase: int = 2
):
    """
    Run text generation on a single dataframe batch and return a list of dicts with predictions.
    """
    # This part is unchanged
    tokenizer.padding_side = "left"
    inputs = tokenizer(
        batch_df["prompt_text"].tolist(),
        return_tensors="pt",
        padding=True,
    ).to(model.device)

    # This part is unchanged
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.9,
            top_p=0.95,
            top_k=20,
        )

    # This part is unchanged
    decoded = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )

    records = []
    # --- THIS IS THE ONLY CHANGE ---
    for idx, full_decoded_string in enumerate(decoded):

        # Your original logic for getting the 'prediction' to be parsed
        prediction_for_parsing = full_decoded_string.split("\nassistant\n")[-1]

        if phase == 1:
            # Your original phase 1 logic, but we add the new raw column
            record = {
                "conversation_number": batch_df.iloc[idx]["conversation_number"],
                "input_text":          batch_df.iloc[idx]["input_text"],
                "prompt_text":         batch_df.iloc[idx]["prompt_text"],
                "current_user_query":  batch_df.iloc[idx]["text"],
                "target_intent":       batch_df.iloc[idx]["intent"],
                "generated_intent":    prediction_for_parsing.strip(),
                "generated_response_raw": full_decoded_string.strip() # The new raw column
            }
            records.append(record)
        else:  # phase == 2
            # Your original phase 2 logic, but we add the new raw column
            record = {
                "conversation_number":  batch_df.iloc[idx]["conversation_number"],
                "text":                 batch_df.iloc[idx]["text"],
                "input_text":           batch_df.iloc[idx]["input_text"],
                "prompt_text":          batch_df.iloc[idx]["prompt_text"],
                "current_user_query":   batch_df.iloc[idx]["current_user_query"],
                "target_intent":       batch_df.iloc[idx]["intent"],
                "target_response":      batch_df.iloc[idx]["assistant_raw"],
                "target_response_int":  batch_df.iloc[idx]["assistant_with_int"],
                "generated_response":   prediction_for_parsing.strip(),
                "generated_response_raw": full_decoded_string.strip() # The new raw column
            }
            records.append(record)

    return records

def run_batched_inference(
    df: pd.DataFrame,
    model,
    tokenizer,
    *,
    batch_size: int = 4,
    max_new_tokens: int = 20,
    phase: int = 2,
    show_progress: bool = True,
) -> pd.DataFrame:
    """High‑level convenience wrapper that iterates **outside** the core batch routine.

    This keeps the inner function small and testable, while allowing you to move the
    outer loop elsewhere (e.g. into a notebook cell or a training script).
    """

    df = df.reset_index(drop=True)

    all_preds = []
    iterator = range(0, len(df), batch_size)
    if show_progress:
        iterator = tqdm(iterator, desc="Generating")

    for start in iterator:
        end = min(start + batch_size, len(df))
        batch_df = df.iloc[start:end]
        batch_records = _infer_single_batch(
            batch_df,
            model,
            tokenizer,
            max_new_tokens=max_new_tokens,
            phase=phase,
        )
        all_preds.extend(batch_records)

    return pd.DataFrame(all_preds)

def strip_think_blocks(text: str) -> str:
    if not isinstance(text, str):
        return text
    # remove full <think>...</think> blocks
    text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    # remove any stray <think> or </think> tags
    text = re.sub(r"</?think>", "", text)
    # collapse multiple newlines into one
    text = re.sub(r"\n+", "\n", text)
    # strip leading/trailing whitespace/newlines
    return text.strip()

In [40]:
df_val_f["text"] = ""
df_val_f["input_text"] = df_val_f["training_text"]
df_val_f["current_user_query"] = ""
df_val_f["assistant_with_int"] = ""

df_val_f.rename({"assistant_intents_only":"intent",
                 "assistant_response_raw":"assistant_raw",
                 "assistant_response_structured":"assistant_with_int"}, axis=1, inplace=True)

output = run_batched_inference(
    df=df_val_f,
    model=model,
    tokenizer=tokenizer,
    batch_size=10,
    max_new_tokens=2048
)
output["generated_response"] = output["generated_response"].apply(strip_think_blocks)
output.head()

Generating:   0%|          | 0/3 [00:00<?, ?it/s]

,conversation_number,text,input_text,prompt_text,current_user_query,target_intent,target_response,target_response_int,generated_response,generated_response_raw
0,18,,<|im_start|>user\nYou are an expert customer s...,<|im_start|>user\nYou are an expert customer s...,,intent clarify_preference intent clarify...,"medewerker: Sure, I can help. Do you want to u...",,Intents:\nUnsubscribe\nResponses:\nResponses:\...,user\nYou are an expert customer service assis...
1,18,,<|im_start|>user\nYou are an expert customer s...,<|im_start|>user\nYou are an expert customer s...,,intent confirm_action intent confirm_act...,medewerker: Understood. I’ve updated your emai...,,"Intents: unsubscribe, confirm unsubscribing, r...",user\nYou are an expert customer service assis...
2,18,,<|im_start|>user\nYou are an expert customer s...,<|im_start|>user\nYou are an expert customer s...,,intent close_conversation intent close_c...,medewerker: Anytime! Your preference is saved.,,Intents: Thank You \nResponses: You're welcom...,user\nYou are an expert customer service assis...
3,19,,<|im_start|>user\nYou are an expert customer s...,<|im_start|>user\nYou are an expert customer s...,,intent request_receipt_number intent req...,medewerker: Sorry about that. Could you share ...,,Intents:\n- Complaint\n- Request for Resolutio...,user\nYou are an expert customer service assis...
4,19,,<|im_start|>user\nYou are an expert customer s...,<|im_start|>user\nYou are an expert customer s...,,intent offer_solution intent...,medewerker: Thanks. I checked and the points d...,,"Intents: Verify Receipt, Confirm Points, Resol...",user\nYou are an expert customer service assis...


## Calculate automatic metrics: BLUE, ROUGE-L, METEOR, BERTScore

In [41]:
import evaluate
import numpy as np
import pandas as pd
from typing import Tuple

def create_response_metrics(
    pred_df: pd.DataFrame,
    *,
    gen_col: str = "pred_reply",
    tgt_col: str = "target_response",
    bert_lang: str = "nl",
    bert_model: str = "xlm-roberta-large",
) -> Tuple[dict, pd.DataFrame]:
    """
    Compute metrics on the FULL dataset only, with **BERTScore only** for embeddings.
    Returns (metrics_dict, enriched_eval_df). Adds per-row BERTScore columns to eval_df.
    """
    # Build a clean eval frame
    eval_df = pd.DataFrame({
        "gen": pred_df[gen_col].fillna("").astype(str),
        "tgt": pred_df[tgt_col].fillna("").astype(str),
    })
    eval_df = eval_df[(eval_df.gen.str.len() > 0) & (eval_df.tgt.str.len() > 0)]
    if eval_df.empty:
        return {"error": "No valid prediction/target pairs."}, eval_df

    # Load metrics
    rouge  = evaluate.load("rouge")
    bleu   = evaluate.load("sacrebleu")
    meteor = evaluate.load("meteor")
    bert   = evaluate.load("bertscore")

    gen, tgt = eval_df.gen.tolist(), eval_df.tgt.tolist()

    # Text overlap metrics (kept as in your previous version)
    r = rouge.compute(predictions=gen, references=tgt)
    b = bleu.compute(predictions=gen, references=[[t] for t in tgt])
    m = meteor.compute(predictions=gen, references=tgt)

    metrics = {
        "bleu_all": b["score"],
        "meteor_all": m["meteor"],
        **{f"{k}_all": v for k, v in r.items()},
    }

    # BERTScore (only embedding-based metric kept)
    bs = bert.compute(predictions=gen, references=tgt, lang=bert_lang, model_type=bert_model)
    for k in ["f1", "precision", "recall"]:
        arr = np.asarray(bs[k], dtype=float)
        metrics[f"bertscore_{k}_all"]     = arr.mean()
        metrics[f"bertscore_{k}_std_all"] = arr.std(ddof=0)

    # Attach per-row BERTScore once
    eval_df["bertscore_f1"]        = bs["f1"]
    eval_df["bertscore_precision"] = bs["precision"]
    eval_df["bertscore_recall"]    = bs["recall"]

    return metrics, eval_df

In [42]:
metrics, eval_df = create_response_metrics(
    output,
    gen_col="generated_response",   # model predictions
    tgt_col="target_response",      # gold responses
    bert_lang="en",                 # or "nl" if Dutch
    bert_model="xlm-roberta-large"
)

print("Aggregate metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

print("\nEval DF with per-row BERTScore:")
print(eval_df.head())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Aggregate metrics:
bleu_all: 0.7967
meteor_all: 0.2243
rouge1_all: 0.1072
rouge2_all: 0.0340
rougeL_all: 0.0966
rougeLsum_all: 0.0986
bertscore_f1_all: 0.8642
bertscore_f1_std_all: 0.0172
bertscore_precision_all: 0.8392
bertscore_precision_std_all: 0.0167
bertscore_recall_all: 0.8910
bertscore_recall_std_all: 0.0229

Eval DF with per-row BERTScore:
                                                 gen  \
0  Intents:\nUnsubscribe\nResponses:\nResponses:\...   
1  Intents: unsubscribe, confirm unsubscribing, r...   
2  Intents: Thank You  \nResponses: You're welcom...   
3  Intents:\n- Complaint\n- Request for Resolutio...   
4  Intents: Verify Receipt, Confirm Points, Resol...   

                                                 tgt  bertscore_f1  \
0  medewerker: Sure, I can help. Do you want to u...      0.876574   
1  medewerker: Understood. I’ve updated your emai...      0.870413   
2     medewerker: Anytime! Your preference is saved.      0.872036   
3  medewerker: Sorry about that.